In [ ]:
import pandas as pd
import numpy as np 
import panel as pn
import datetime as dt
import param
pn.extension('tabulator')

import hvplot.pandas

<h3>Preparing Data for Plotting</h3>

In [ ]:
# Importing Csv with data
df = pd.read_csv('/Users/uzytkownik/desktop/border_traffic_UA_PL_01_03.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df = df.fillna(0)

In [ ]:
df.rename(columns = {'Border crossing' : 'Border crossing name',
                     'Type of border crossing' : 'Type of border',
                     'Direction to / from Poland' : 'Direction', 'Citizenship (code)' : 'Citizenship_code'}, inplace=True) 

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format = '%Y/%m/%d')

In [ ]:
df['Month'] = df['Date'].dt.strftime('%m')

In [ ]:
def month_name(x):
    if x == '01':
        return 'January'
    elif x == '02':
        return 'February'
    else:
        return 'March'

In [ ]:
df['Month_name'] = df['Month'].apply(month_name)

In [ ]:
df['Border Guard Post'].unique()

In [ ]:
df.sort_values(by = 'Date', ascending = True, inplace = True)

In [ ]:
df['Month'] = df['Month'].astype(int)

In [ ]:
def func_zero(x):
    if x == '0':
        return 'Other'
    elif x == 'UE':
        return 'European Union'
    else:
        return 'Schengen'

In [ ]:
df['UE / Schengen'] = df['UE / Schengen'].apply(func_zero)

In [ ]:
df['Total number of people crossing border'] = df['Number of persons (checked-in)']+df['Number of people (evacuated)']

<h3>Visualizations</h3>

In [ ]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [ ]:
# buttons for Border crossing name (plot 1)
radio_group1 = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['Number of persons (checked-in)','Number of people (evacuated)'],
    button_type = 'primary'
)

In [ ]:
# (1)Timeseries 1
border_pipeline = (
    idf[
        (idf.Date <= df['Date']) &
        (idf.Citizenship_code.isin(df['Citizenship_code']))
    ]
    .groupby(['Citizenship_code','Date'])[radio_group1].sum()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Date')
    .reset_index(drop = True)
)

In [ ]:
# (1)Timeseries 1
border_timeseries1 = border_pipeline.hvplot(x = 'Date', by = 'Citizenship_code', y = radio_group1, line_width = 2,title = 'Number of border crossings by Cirizenship code')

In [ ]:
border_timeseries1

In [ ]:
# (2)Table
border_table_data = (idf[['Citizenship_code','Total number of people crossing border']]
                     .groupby(by ='Citizenship_code').sum()
                     .reset_index()
                     .sort_values(by= 'Total number of people crossing border', ascending=False) 
                     .reset_index(drop = True)
                    )

In [ ]:
# (2)Table
border_table = border_table_data.pipe(pn.widgets.Tabulator, page_size = 10, pagination = 'remote', sizing_mode = 'stretch_width')

In [ ]:
border_table

In [ ]:
# (3)Timeseries 2
border_pipeline2 = (
    idf[['Date','Direction','Total number of people crossing border']]
    .groupby(['Date','Direction']).sum()
    .reset_index()
    .sort_values(by = 'Date', ascending = True)
    .reset_index(drop = True)
)

In [ ]:
# (3)Timeseries 2
border_timeseries2 = border_pipeline2.hvplot( x = 'Date', by = 'Direction', y = 'Total number of people crossing border', line_width = 2, title = 'Number of border crossings by Direction', legend = 'top_left')

In [ ]:
border_timeseries2

In [ ]:
# (4) Bar chart 
border_pipeline3 = (
    idf[['UE / Schengen','Total number of people crossing border',
         'Number of persons (checked-in)',
         'Number of people (evacuated)']]
    .groupby(['UE / Schengen']).sum()
    .reset_index()
    .sort_values(by = 'Total number of people crossing border', ascending = False)
    .reset_index(drop = True)
)

In [ ]:
# (4) Bar chart 
border_bar = border_pipeline3.hvplot(kind ='bar',
                                      x = 'UE / Schengen',
                                      y = ['Number of persons (checked-in)','Number of people (evacuated)'],
                                      stacked = True,
                                      title = 'Total number of people crossing border', 
                                      legend='top_right', 
                                      ylabel= 'Total number of people crossing border',
                                      xlabel = 'Membership')

In [ ]:
border_bar

In [ ]:
# (5) Barh chart
border_pipeline4 = (
    idf[['Month_name','Month','Border Guard Post','Total number of people crossing border']]
    .groupby(['Month','Month_name','Border Guard Post']).sum()
    .reset_index()
    .sort_values(by = 'Month', ascending = False)
    .reset_index(drop = True)
)

In [ ]:
# (5) Barh chart
border_barh = border_pipeline4.hvplot(kind = 'barh',
                                      x = 'Month_name',
                                      y = 'Total number of people crossing border',
                                      by = 'Border Guard Post',
                                      stacked = False,
                                      title = 'Number of border crossings by Guard Post Name',  
                                      ylabel = 'Guard Post Name',
                                      xlabel = 'Month')
      

In [ ]:
border_barh

<h3>Creating Dashboard</h3>

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title= 'Polish-Ukrainian border traffic dashboard', 
    sidebar=[pn.pane.Markdown("# Traffic on Polish-Ukrainian border recorded from Jan 2022 to Mar 2022"), 
             pn.pane.Markdown("#### Dashboard was created based on dataset published on Kaggle. The Dataset containing statistical data refers to the situation on the Polish-Ukrainian border between January and April 2022, including (among others): Number of people evacuated, checked in, Citizenship code of people crossing border and Guard Border Post name."), 
             pn.pane.PNG('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSSFCS9WeMESgtLlx-yOB5mq1OIVOfWhSRIsqaGKkNiaTRzwEy_LNY_UOjonB9StHrnKv0&usqp=CAU.png', sizing_mode='scale_both')],
    main=[pn.Row(pn.Column(radio_group1, 
                           border_timeseries1.panel(width=700), margin=(0,25)), 
                 pn.Column(border_table.panel(width=590))), 
          pn.Row(pn.Column(border_timeseries2.panel(width=700), margin=(0,25)), 
                 pn.Column(border_bar.panel(width=600))),
          pn.Row(border_barh.panel(width=1300, height = 400))],
    accent_base_color="#1874CD",
    header_background="#104E8B",
)
# template.show()
template.servable();